In [1]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
# importa modulos
import pickle
import numpy as np
import itertools
import pandas as pd
import mtspec as sp
from scipy import signal
import matplotlib
import matplotlib.pyplot as plt
import time

In [2]:
# RAT01
rat = 'RAT01'
ruta = '/home/romulo/DATA/Carolina/RR01/EC01_RRR01_SP_12600uA__20180103/'
fname = 'LFP_EC01_RRR01_SP_12600uA__20180103'

# RAT02
#ruta = '/home/romulo/DATA/Carolina/R02/RAT02_SP_7_2017_01_11_18_46_49/'
#rat='RAT02'
#ruta='/home/romulo/DATA/'
#fname = 'LFP_RAT02_SP_7_2017_01_11_18_46_49'

# RAT03
#rat='RAT03'
#ruta = '/home/romulo/DATA/Carolina/RR03/EC01_RRR03_SP_3500uA_20171215/'
#fname = 'LFP_EC01_RRR03_SP_3500uA_20171215'

#rat 05
#rat = 'RAT05'
#fname='LFP_RC05_SP_10100uA_20171214_170009'
#ruta='/home/romulo/DATA/Carolina/C05/EC01_RC05_SP_10100uA_20171214/'

data = pickle.load(open(ruta+fname+'.pickle', "rb" ) )
lfp = data[0]

In [3]:
lfp

,Channel,Area,OriginalSampleRate,SampleRate,Use,Timestamps at original sample rate,LFP
0,CH1,rVL,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[0.258195505622, 0.386687295735, 0.53261572248..."
1,CH2,rVL,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[-0.135889537346, -0.0866598819283, -0.0613932..."
2,CH3,rSTR,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[0.0536468101846, 0.0824154562411, 0.042028277..."
3,CH4,rSTR,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[-0.0581481118995, 0.0556404239133, -0.0159933..."
4,CH5,rSTR,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[0.0853395658702, -0.209062241342, 0.041582291..."
5,CH6,rM1,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[-0.0435634288454, -0.0573218765439, 0.0547644..."
6,CH7,rM1,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[-0.161651675978, -0.273899652084, -0.19464382..."
7,CH8,rM1,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[-0.00125280059258, -0.0043022890135, -0.05226..."
8,CH9,rM1,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[0.379472827914, 0.434236442851, 0.32369588637..."
9,CH10,rM1,30000,1000,1,"[28701000, 28702024, 28703048, 28704072, 28705...","[-0.364692766244, -0.24370295928, -0.216456263..."


In [6]:
# Extraer canal con los pulsos de estimulacion, siempre será el canal ADC8 
import OpenEphys as OE

ttl_channel = '100_ADC8' 
extension = '.continuous'
ttl_channel = ttl_channel+extension
data = OE.loadContinuous(ruta+ttl_channel)
ttl_signal = data['data']
sampleRate = np.double(data['header']['sampleRate'])
# construye un vector de tiempo con el samplerate
t0 = 1/sampleRate
tf = len(ttl_signal)/sampleRate + 1/sampleRate
step = 1/sampleRate
ttl_t = np.arange(t0,tf,step)
# normaliza la señal de ttl, minimo 0, maximo 1
delta = max(ttl_signal)- min(ttl_signal)
ttl_signal = (ttl_signal - min(ttl_signal))/delta
# busca los indices de las muestras > o iguales a 0.8 (tolerancia de 0.2)
thr = 0.7
ind = ttl_signal>=thr
# encuentra los tiempos
time_stamps = ttl_t[ind]

flag = 0
on_ts = np.empty(0)
off_ts = np.empty(0)
for i in range(0,len(ttl_signal)):
    if flag == 0:
        if ttl_signal[i]>thr:
            on_ts = np.append(on_ts,ttl_t[i])
            flag = 1
    if flag == 1:
        if ttl_signal[i]<thr:
            off_ts = np.append(off_ts,ttl_t[i])
            flag = 0  
len(on_ts)

Loading continuous data from /home/romulo/DATA/Carolina/RR01/EC01_RRR01_SP_12600uA__20180103/100_ADC8.continuous


1000

In [4]:
# load stimulation timestamps
#stimfile = 'STIM_RAT02_SP_7_2017_01_11_18_46_49'
#stimfile='STIM_EC01_RRR03_SP_3500uA_20171215'
#data = pickle.load(open(ruta+stimfile+'.pickle', "rb" ) )
data = pickle.load(open(ruta+'STIM_'+fname[4:]+'.pickle', "rb" ) )
on_ts = data[0]
del data

In [8]:
# save stimulation timestamps
pickle.dump([on_ts],open(ruta+'STIM_' + fname[4:] +'.pickle', "wb" ) )

# Compute Evoked potential and it spectrogram and save to dataframe, create and save figure 

In [9]:
Ldopa = 'off'
N = 2
sr = lfp.SampleRate[0]
nyq = sr/2.0
f1 = 0.1
f2 = 75.0
sr = lfp.SampleRate[0] # # extract sampling rate and...
t = np.arange(1/sr,len(lfp.LFP[0])/sr + 1/sr,1/sr) #create a time axis t
past = 0.3
future = 0.7
tt = np.arange(-past,future,1/sr)

# filter parameters
Wn = np.array([f1/nyq, f2/nyq]) # band pass freqs expressed as Nyquist
b,a = signal.butter(N, Wn, btype='bandpass', analog=False) # design parameters for the butter filter

# spectrogram parameters
delta=1/sr
time_bandwidth=2.0
maxf=75.0
windowsize = 0.07 # in seconds
windowsize = int(windowsize*sr)
stepsize = 1
leninsig = 2**11 # length input signal 2**11
numloops = int(len(tt)/stepsize - windowsize) # for stepsize of 1 sample
power= sp.mtspec(np.random.rand(leninsig),delta,time_bandwidth)
f=power[1]
f=f[f<maxf]

df = pd.DataFrame([], columns=['Rat',
                               'File',
                               'L-dopa',
                               'SampleRate',
                               'Bandpass',
                               'tWindow',
                               'Channel',
                               'Area',
                               'EvokedPotential',
                               'EvokedPotSpec', 
                               'EvokedPotSpecZ',
                               'SpecPerturbation'])
for i in range(0,len(lfp)):
    ad = signal.filtfilt(b, a, lfp.LFP.loc[i], padlen=None)
    lfparray = np.zeros([len(on_ts),int(past*sr+future*sr)])
    out = np.zeros(len(on_ts))

    # collect the trials around stimulation pulse
    for j in range(0,len(on_ts)):
        t1 = on_ts[j]
        ind = int(sr*round(t1,3)-1)
        adaux = ad[int(ind-past*sr):int(ind+future*sr)]
        
        lfparray[j,:] = adaux
        out[j]=np.mean(np.abs(np.diff(adaux)))
    sorted_out = np.sort(out)
    lim =  sorted_out[int(len(sorted_out)-len(sorted_out)*5/100)]
    lfparray=lfparray[out<lim,:] 
    evpot = np.mean(lfparray,0)
    ttt = np.zeros(numloops)
    S = np.zeros([len(f),numloops])
    for j in range(0,numloops):
        aux = evpot[j*stepsize:j*stepsize+windowsize]
        ttt[j] = np.mean(tt[j*stepsize:j*stepsize+windowsize])
        input2psd = np.zeros(leninsig)
        ind2target = int(leninsig/2) - int(len(aux)/2)
        input2psd[ind2target:ind2target+len(aux)] =  aux
        power= sp.mtspec(input2psd,delta,time_bandwidth)
        f=power[1]
        power=power[0]
        S[:,j]=power[f<maxf]
    f=f[f<maxf]
    baseave = np.mean(np.log(S[:,ttt<-0.01]),1)
    basestd = np.std(np.log(S[:,ttt<-0.01]),1)
    aveS = np.tile(baseave,(numloops,1)).T
    stdS = np.tile(basestd,(numloops,1)).T
    SS = (np.log(S)-aveS) / stdS
    
    fig = plt.figure()
    ax=plt.subplot(211)
    ax.plot(tt,evpot)
    ax.set_title(lfp.Channel[i]+'-'+lfp.Area[i])
    ax.set_xlim(-past, future)
    ax.set_xticks(np.arange(-past, future, 0.1))
    ax.set_xticklabels(range(int(-past*1000),int(future*1000),100))
    ax.grid()
    ax.set_ylabel('Potential[z-scored]')
    
    ax=plt.subplot(212)
    ax.pcolormesh(ttt,f,SS,vmin=-2.5, vmax=2.5)
    ax.plot(np.zeros(len(f)),np.linspace(0, maxf, len(f)),'w')
    ax.set_xlim(-past, future)
    ax.set_ylim(0, 75)
    ax.set_xticks(np.arange(-past, future, 0.1))
    ax.set_xticklabels(range(int(-past*1000),int(future*1000),100))
    ax.set_yticks(range(0,int(maxf),5))
    ax.set_xlabel('Time[ms]')
    ax.set_ylabel('Frequency[Hz]')
    figname = 'EvokedPot_Spectr_'+rat+'_'+ lfp.Channel[i] + '-' + lfp.Area[i]+ '.png'
    print figname 
    #plt.show(block=False)
    fig.savefig(ruta+figname)
    plt.close(fig)
    #break
    df2 = pd.DataFrame([[rat,
                         fname,
                         Ldopa,
                         lfp.SampleRate.loc[i],
                         [f1,f2],
                         [-past,future],
                         lfp.Channel.loc[i],
                         lfp.Area.loc[i],
                         lfparray.mean(0),
                         S,
                         SS,
                         []]],
                       columns=['Rat',
                               'File',
                               'L-dopa',
                               'SampleRate',
                               'Bandpass',
                               'tWindow',
                               'Channel',
                               'Area',
                               'EvokedPotential',
                               'EvokedPotSpec', 
                               'EvokedPotSpecZ',
                               'SpecPerturbation'])
    df = pd.concat([df, df2], axis = 0)
df=df.reset_index()
del df['index']

EvokedPot_Spectr_RAT01_CH1-rVL.png
EvokedPot_Spectr_RAT01_CH2-rVL.png
EvokedPot_Spectr_RAT01_CH3-rSTR.png
EvokedPot_Spectr_RAT01_CH4-rSTR.png
EvokedPot_Spectr_RAT01_CH5-rSTR.png
EvokedPot_Spectr_RAT01_CH6-rM1.png
EvokedPot_Spectr_RAT01_CH7-rM1.png
EvokedPot_Spectr_RAT01_CH8-rM1.png
EvokedPot_Spectr_RAT01_CH9-rM1.png
EvokedPot_Spectr_RAT01_CH10-rM1.png
EvokedPot_Spectr_RAT01_CH11-rM1.png
EvokedPot_Spectr_RAT01_CH12-rSTR.png
EvokedPot_Spectr_RAT01_CH13-rSTR.png
EvokedPot_Spectr_RAT01_CH14-rSTR.png
EvokedPot_Spectr_RAT01_CH15-rSTR.png
EvokedPot_Spectr_RAT01_CH16-rVL.png
EvokedPot_Spectr_RAT01_CH17-rVPL.png
EvokedPot_Spectr_RAT01_CH18-rVPL.png
EvokedPot_Spectr_RAT01_CH20-rS1.png
EvokedPot_Spectr_RAT01_CH21-rSTN.png
EvokedPot_Spectr_RAT01_CH22-rSTN.png
EvokedPot_Spectr_RAT01_CH23-rPf.png
EvokedPot_Spectr_RAT01_CH24-rPf.png
EvokedPot_Spectr_RAT01_CH25-rPf.png
EvokedPot_Spectr_RAT01_CH26-rPf.png
EvokedPot_Spectr_RAT01_CH27-rSTN.png
EvokedPot_Spectr_RAT01_CH28-rS1.png
EvokedPot_Spectr_RAT01_CH

### Compute Evoked potential and its spectrogram, and the spectral perturbation and save all to dataframe, create and save figure

In [6]:
# 'LFP_RAT02_SP_7_2017_01_11_18_46_49'
#DontUseChannels = ['CH10', 'CH21', 'CH22', 'CH23', 'CH24', 'CH35', 'CH36']
# LFP_RC05_SP_10100uA_20171214_170009
# DontUseChannels = ['CH14', 'CH15']
# LFP_EC01_RRR01_SP_12600uA__20180103
DontUseChannels = []

In [7]:
timer = time.time()
Ldopa = 'off'
N = 2
sr = lfp.SampleRate[0]
nyq = sr/2.0
f1 = 0.1
f2 = 75.0
sr = lfp.SampleRate[0] # # extract sampling rate and...
t = np.arange(1/sr,len(lfp.LFP[0])/sr + 1/sr,1/sr) #create a time axis t
past = 0.3
future = 0.7
tt = np.arange(-past,future,1/sr)

# filter parameters
Wn = np.array([f1/nyq, f2/nyq]) # band pass freqs expressed as Nyquist
b,a = signal.butter(N, Wn, btype='bandpass', analog=False) # design parameters for the butter filter

# spectrogram parameters
delta=1/sr
time_bandwidth=2.0
maxf=75.0
windowsize = 0.07 # in seconds
windowsize = int(windowsize*sr)
stepsize = 1
leninsig = 2**11 # length input signal 2**11
numloops = int(len(tt)/stepsize - windowsize) # for stepsize of 1 sample
power= sp.mtspec(np.random.rand(leninsig),delta,time_bandwidth)
f=power[1]
ff=f[f<maxf]

df = pd.DataFrame([], columns=['Rat',
                               'File',
                               'L-dopa',
                               'SampleRate',
                               'Bandpass',
                               'tWindow',
                               'Channel',
                               'Area',
                               'EvokedPotential',
                               'EvokedPotSpec', 
                               'EvokedPotSpecZ',
                               'SpecPerturbation'])
for i in range(0,len(lfp)):
    if lfp.Channel[i] not in DontUseChannels:
        ad = signal.filtfilt(b, a, lfp.LFP.loc[i], padlen=None)
        lfparray = np.zeros([len(on_ts),int(past*sr+future*sr)])
        out = np.zeros(len(on_ts))

        # collect the trials around stimulation pulse
        for j in range(0,len(on_ts)):
            t1 = on_ts[j]
            ind = int(sr*round(t1,3)-1)
            adaux = ad[int(ind-past*sr):int(ind+future*sr)]

            lfparray[j,:] = adaux
            out[j]=np.mean(np.abs(np.diff(adaux)))
        sorted_out = np.sort(out)
        lim =  sorted_out[int(len(sorted_out)-len(sorted_out)*5/100)]
        lfparray=lfparray[out<lim,:] 
        
        # spectrogram of the evoked potential (spectrogram of the average)
        evpot = np.mean(lfparray,0)
        ttt = np.zeros(numloops)
        S = np.zeros([len(ff),numloops])
        for j in range(0,numloops):
            aux = evpot[j*stepsize:j*stepsize+windowsize]
            ttt[j] = np.mean(tt[j*stepsize:j*stepsize+windowsize])
            input2psd = np.zeros(leninsig)
            ind2target = int(leninsig/2) - int(len(aux)/2)
            input2psd[ind2target:ind2target+len(aux)] =  aux
            power= sp.mtspec(input2psd,delta,time_bandwidth)
            f=power[1]
            power=power[0]
            S[:,j]=power[f<maxf]
        f=f[f<maxf]
        baseave = np.mean(np.log(S[:,ttt<-0.01]),1)
        basestd = np.std(np.log(S[:,ttt<-0.01]),1)
        aveS = np.tile(baseave,(numloops,1)).T
        stdS = np.tile(basestd,(numloops,1)).T
        SS = (np.log(S)-aveS) / stdS

        # spectral perturbation (average spectrogram)
        mSP = np.zeros([len(ff),numloops,np.shape(lfparray)[0]])
        for k in range(0,np.shape(lfparray)[0]):
            SP = np.zeros([len(ff),numloops])
            for j in range(0,numloops):
                aux = lfparray[k, j*stepsize:j*stepsize+windowsize]
                input2psd = np.zeros(leninsig)
                ind2target = int(leninsig/2) - int(len(aux)/2)
                input2psd[ind2target:ind2target+len(aux)] =  aux
                power= sp.mtspec(input2psd,delta,time_bandwidth)
                f=power[1]
                power=power[0]
                SP[:,j]=power[f<maxf]
            mSP[:,:,k] = SP
        SP=np.mean(np.log(mSP), axis=2)
        
        fig = plt.figure(figsize=([7, 9]))
        
        ax=plt.subplot(311)
        ax.plot(tt,evpot)
        ax.set_title('Evoked Potential ' + lfp.Channel[i]+'-'+lfp.Area[i])
        ax.set_xlim(-past, future)
        ax.set_xticks(np.arange(-past, future, 0.1))
        ax.set_xticklabels(range(int(-past*1000),int(future*1000),100))
        ax.grid()
        ax.set_ylabel('Potential[z-scored]')

        ax=plt.subplot(312)
        ax.pcolormesh(ttt,ff,SS,vmin=-2.5, vmax=2.5)
        ax.plot(np.zeros(len(f)),np.linspace(0, maxf, len(f)),'w')
        ax.set_xlim(-past, future)
        ax.set_ylim(0, maxf)
        ax.set_yticks(range(0,int(maxf),5))
        ax.set_xticks(np.arange(-past, future, 0.1))
        ax.set_xticklabels(range(int(-past*1000),int(future*1000),100))
        #ax.set_title('Spectrogram of Evoked Potential')
        ax.set_ylabel('Frequency[Hz]')

        ax=plt.subplot(313)
        ax.pcolormesh(ttt,ff,SP)
        ax.plot(np.zeros(len(f)),np.linspace(0, maxf, len(f)),'w')
        ax.set_xlim(-past, future)
        ax.set_ylim(0, maxf)
        ax.set_yticks(range(0,int(maxf),5))
        #ax.set_title('Spectral Perturbation')

        ax.set_xticks(np.arange(-past, future, 0.1))
        ax.set_xticklabels(range(int(-past*1000),int(future*1000),100))

        ax.set_xlabel('Time[ms]')
        ax.set_ylabel('Frequency[Hz]')
        
        figname = 'EvokedPot_Spectr_'+rat+'_'+ lfp.Channel[i] + '-' + lfp.Area[i]+ '.png'
        print figname 
        #plt.show(block=False)
        fig.savefig(ruta+figname)
        plt.close(fig)
        df2 = pd.DataFrame([[rat,
                             fname,
                             Ldopa,
                             lfp.SampleRate.loc[i],
                             [f1,f2],
                             [-past,future],
                             lfp.Channel.loc[i],
                             lfp.Area.loc[i],
                             lfparray.mean(0),
                             S,
                             SS,
                             SP]],
                           columns=['Rat',
                                   'File',
                                   'L-dopa',
                                   'SampleRate',
                                   'Bandpass',
                                   'tWindow',
                                   'Channel',
                                   'Area',
                                   'EvokedPotential',
                                   'EvokedPotSpec', 
                                   'EvokedPotSpecZ',
                                   'SpecPerturbation'])
        df = pd.concat([df, df2], axis = 0)
df=df.reset_index()
del df['index']
elapsed = time.time() - timer
print elapsed

# save to pickle
df.to_pickle(ruta+'EvokedPot_'+fname+'.pickle')
print 'Saving file to '+ ruta+'EvokedPot_'+fname+'.pickle'
# save axis
pickle.dump([ttt,ff],open(ruta+'TimeFrequencyAxis.pickle','wb'))

EvokedPot_Spectr_RAT01_CH1-rVL.png
EvokedPot_Spectr_RAT01_CH2-rVL.png
EvokedPot_Spectr_RAT01_CH3-rSTR.png
EvokedPot_Spectr_RAT01_CH4-rSTR.png
EvokedPot_Spectr_RAT01_CH5-rSTR.png
EvokedPot_Spectr_RAT01_CH6-rM1.png
EvokedPot_Spectr_RAT01_CH7-rM1.png
EvokedPot_Spectr_RAT01_CH8-rM1.png
EvokedPot_Spectr_RAT01_CH9-rM1.png
EvokedPot_Spectr_RAT01_CH10-rM1.png
EvokedPot_Spectr_RAT01_CH11-rM1.png
EvokedPot_Spectr_RAT01_CH12-rSTR.png
EvokedPot_Spectr_RAT01_CH13-rSTR.png
EvokedPot_Spectr_RAT01_CH14-rSTR.png
EvokedPot_Spectr_RAT01_CH15-rSTR.png
EvokedPot_Spectr_RAT01_CH16-rVL.png
EvokedPot_Spectr_RAT01_CH17-rVPL.png
EvokedPot_Spectr_RAT01_CH18-rVPL.png
EvokedPot_Spectr_RAT01_CH20-rS1.png
EvokedPot_Spectr_RAT01_CH21-rSTN.png
EvokedPot_Spectr_RAT01_CH22-rSTN.png
EvokedPot_Spectr_RAT01_CH23-rPf.png
EvokedPot_Spectr_RAT01_CH24-rPf.png
EvokedPot_Spectr_RAT01_CH25-rPf.png
EvokedPot_Spectr_RAT01_CH26-rPf.png
EvokedPot_Spectr_RAT01_CH27-rSTN.png
EvokedPot_Spectr_RAT01_CH28-rS1.png
EvokedPot_Spectr_RAT01_CH

Saving file to /home/romulo/DATA/Carolina/C05/EC01_RC05_SP_10100uA_20171214/EvokedPot_LFP_RC05_SP_10100uA_20171214_170009.pickle
